In [12]:
# char-rnn.py
# Neural Character Language Model in CNTK2
# wdarling@microsoft.com

import numpy as np
import os
from cntk import Trainer, Axis
from cntk.learner import adam_sgd, momentum_sgd, momentum_as_time_constant_schedule, learning_rate_schedule, UnitType
from cntk.ops import input_variable, cross_entropy_with_softmax, classification_error
from cntk.persist import load_model, save_model
from cntk.blocks import LSTM, Stabilizer
from cntk.layers import Recurrence, Dense, Dropout, BatchNormalization
from cntk.utils import get_train_eval_criterion, get_train_loss
from cntk.device import set_default_device, gpu

# Set to GPU, run if GPU is available
set_default_device(gpu(0))

# model hyperparameters
hidden_dim = 256
num_layers = 2
minibatch_size = 100 # also how much time we unroll the RNN for

# Get data
def get_data(p, minibatch_size, data, char_to_ix, vocab_dim):

    xi = [char_to_ix[ch] for ch in data[p:p+minibatch_size]]
    yi = [char_to_ix[ch] for ch in data[p+1:p+minibatch_size+1]]
    
    X = np.eye(vocab_dim, dtype=np.float32)[xi]
    Y = np.eye(vocab_dim, dtype=np.float32)[yi]

    # return a list of numpy arrays for each of X (features) and Y (labels)
    return [X], [Y]

# Sample from the network
def sample(root, ix_to_char, vocab_dim, char_to_ix, prime_text='', use_hardmax=True, length=100, temperature=1.0):

    # temperature: T < 1 means smoother; T=1.0 means same; T > 1 means more peaked
    def apply_temp(p):
        # apply temperature
        p = np.power(p, (temperature))
        # renormalize and return
        return (p / np.sum(p))

    def sample_word(p):
        if use_hardmax:
            w = np.argmax(p, axis=2)[0,0]
        else:
            # normalize probabilities then take weighted sample
            p = np.exp(p) / np.sum(np.exp(p))            
            p = apply_temp(p)
            w = np.random.choice(range(vocab_dim), p=p.ravel())
        return w

    plen = 1
    prime = -1

    # start sequence with first input    
    x = np.zeros((1, vocab_dim), dtype=np.float32)    
    if prime_text != '':
        plen = len(prime_text)
        prime = char_to_ix[prime_text[0]]
    else:
        prime = np.random.choice(range(vocab_dim))
    x[0, prime] = 1
    arguments = ([x], [True])

    output=[]
    output.append(prime)
    
    # loop through prime text
    for i in range(plen):            
        p = root.eval(arguments)        
        
        # reset
        x = np.zeros((1, vocab_dim), dtype=np.float32)
        if i < plen-1:
            idx = char_to_ix[prime_text[i+1]]
        else:
            idx = sample_word(p)

        output.append(idx)
        x[0, idx] = 1            
        arguments = ([x], [False])
    
    # loop through length of generated text, sampling along the way
    for i in range(length-plen):
        p = root.eval(arguments)
        idx = sample_word(p)
        output.append(idx)

        x = np.zeros((1, vocab_dim), dtype=np.float32)
        x[0, idx] = 1
        arguments = ([x], [False])

    # return output
    return ''.join([ix_to_char[c] for c in output])

def load_data_and_vocab(training_file):
    
    # load data
    rel_path = training_file
    path = rel_path
    data = open(path, "r", encoding='utf8').read()
    chars = sorted(list(set(data)))
    data_size, vocab_size = len(data), len(chars)
    print('data has %d characters, %d unique.' % (data_size, vocab_size))
    char_to_ix = { ch:i for i,ch in enumerate(chars) }
    ix_to_char = { i:ch for i,ch in enumerate(chars) }

    # write vocab
    ff = open(path + ".vocab", "w")
    for c in chars:
        ff.write("%s\n" % c) if c != '\n' else ff.write("\n")
    ff.close()
    
    return data, char_to_ix, ix_to_char, data_size, vocab_size

# Creates and trains a character-level language model
def train_lm(training_file):

    # create the stabilizer function from blocks
    stabilize = Stabilizer()

    # load the data and vocab
    data, char_to_ix, ix_to_char, data_size, vocab_dim = load_data_and_vocab(training_file)

    # Source and target inputs to the model
    batch_axis = Axis.default_batch_axis()
    input_seq_axis = Axis('inputAxis')

    input_dynamic_axes = [batch_axis, input_seq_axis]
    raw_input = input_variable(shape=(vocab_dim), dynamic_axes=input_dynamic_axes)
    raw_labels = input_variable(shape=(vocab_dim), dynamic_axes=input_dynamic_axes)

    input_sequence = raw_input
    label_sequence = raw_labels

    # LSTM
    encoder_output = stabilize(input_sequence)
    for i in range(0, num_layers):
        encoder_output = Recurrence(LSTM(hidden_dim, enable_self_stabilization=True)) (encoder_output.output)
        encoder_output = BatchNormalization() (encoder_output.output)
        encoder_output = Dropout(0.5) (encoder_output.output)

    # get output of the LSTM
    states = encoder_output.output

    # dense layer    
    z = Dense(vocab_dim) (states)

    ce = cross_entropy_with_softmax(z, label_sequence)
    errs = classification_error(z, label_sequence)

    # Instantiate the trainer object to drive the model training
    lr_per_sample = learning_rate_schedule(0.005, UnitType.sample)
    momentum_time_constant = momentum_as_time_constant_schedule(1100)
    clipping_threshold_per_sample = 5.0
    gradient_clipping_with_truncation = True
    learner = adam_sgd(z.parameters, lr_per_sample, momentum_time_constant, 
                           gradient_clipping_threshold_per_sample=clipping_threshold_per_sample,
                           gradient_clipping_with_truncation=gradient_clipping_with_truncation)
    trainer = Trainer(z, ce, errs, learner)

    training_progress_output_freq = 100
    sample_freq = 1000
    epochs = 50
    minibatches_per_epoch = int((data_size / minibatch_size))
    minibatches = epochs * minibatches_per_epoch
    
    e = 0
    p = 0
    for i in range(0, minibatches):

        if p + minibatch_size+1 >= data_size:
            p = 0
            e += 1
            model_filename = "models/deepjingling-songwriter_epoch%d.dnn" % e
            save_model(z, model_filename)
            print("Saved model to '%s'" % model_filename)

        # get the data            
        features, labels = get_data(p, minibatch_size, data, char_to_ix, vocab_dim)

        # Specify the mapping of input variables in the model to actual minibatch data to be trained with
        # If it's the start of the data, we specify that we are looking at a new sequence (True)
        mask = [False] 
        if p == 0:
            mask = [True]
        arguments = ({raw_input : features, raw_labels : labels}, mask)
        trainer.train_minibatch(arguments)

        if i % training_progress_output_freq == 0:
            print("Minibatch: {}, Train Loss: {}, Train Evaluation Criterion: {}".format(i,
                      get_train_loss(trainer), get_train_eval_criterion(trainer)))
            print("Epoch %d, %f %% done" % (e, ((float(i) / float(minibatches_per_epoch)) - e) * 100.0))
        
        if i % sample_freq == 0:
            print(sample(z, ix_to_char, vocab_dim, char_to_ix))

        p += minibatch_size
        

def load_and_sample(model_filename, vocab_filename, prime_text='', use_hardmax=False, length=1000, temperature=1.0):
    
    # load the model
    model = load_model(model_filename)
    
    # load the vocab
    chars = [c[0] for c in open(vocab_filename).readlines()]
    char_to_ix = { ch:i for i,ch in enumerate(chars) }
    ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
    #output = sample(model, ix_to_char, len(chars), char_to_ix, prime_text=prime_text, use_hardmax=use_hardmax, length=length, temperature=temperature)
    
    print(sample(model, ix_to_char, len(chars), char_to_ix))

    #ff = open('output.txt', 'w', encoding='utf-8')
    #ff.write(output)
    #ff.close()

if __name__=='__main__':    
    
    # train the LM
    train_lm("data/songs.txt")

    # load and sample
    #text = "T"
    #load_and_sample("models/shakespeare_epoch13.dnn", "shakespeare.txt.vocab", prime_text=text, use_hardmax=False, length=100, temperature=1.0)


RuntimeError: Process wide default device cannot be changed since it has been frozen by being implicitly used as the default device in a CNTK API call